# TAPATAN

## STANDARD FORMULATION

In [1]:
import sys
import numpy as np
from IPython.display import clear_output

sys.path.append('../')

from tapatan.constants import BLACK, WHITE
from tapatan.logic_game import TapatanGrid
from tapatan.formulations import new_grid, available_moves_user, move, win

In [2]:
# Declaring auxiliary functions

def grid2board(grid):
    return '  -  '.join(map(str, grid[0])) + '\n' + \
           '|  \  |  /  |'                 + '\n' + \
           '  -  '.join(map(str, grid[1])) + '\n' + \
           '|  /  |  \  |'                 + '\n' + \
           '  -  '.join(map(str, grid[2]))

def get_new_state(state, user, start, final):
    new_state = state.copy()
    
    new_state[start] = 0
    new_state[final] = user

    return new_state

# Declaring the evaluation function
# MAX -> White pieces
# MIN -> Black pieces

def evaluate(state):
    score = 0
    
    if win(state, WHITE):
        score = +1
    elif win(state, BLACK):
        score = -1

    return score

In [3]:
# Testing the evaluation function

grid_win  = new_grid()
grid_lose = new_grid()
grid_tie  = new_grid()

move(grid_win, BLACK, (0, 0), (0, 1))
move(grid_win, WHITE, (2, 2), (1, 1))
move(grid_win, BLACK, (2, 0), (2, 1))
move(grid_win, WHITE, (1, 0), (2, 0))

move(grid_lose, BLACK, (2, 0), (1, 1))
move(grid_lose, WHITE, (1, 0), (2, 0))
move(grid_lose, BLACK, (0, 0), (1, 0))


print('Expected evaluation = 1')
print('Evaluation obtained =', evaluate(grid_win))
print(grid2board(grid_win))

print()

print('Expected evaluation = 0')
print('Evaluation obtained =', evaluate(grid_tie))
print(grid2board(grid_tie))

print()

print('Expected evaluation = -1')
print('Evaluation obtained =', evaluate(grid_lose))
print(grid2board(grid_lose))

Expected evaluation = 1
Evaluation obtained = 1
0  -  1  -  2
|  \  |  /  |
0  -  2  -  1
|  /  |  \  |
2  -  1  -  0

Expected evaluation = 0
Evaluation obtained = 0
1  -  0  -  2
|  \  |  /  |
2  -  0  -  1
|  /  |  \  |
1  -  0  -  2

Expected evaluation = -1
Evaluation obtained = -1
0  -  0  -  2
|  \  |  /  |
1  -  1  -  1
|  /  |  \  |
2  -  0  -  2


## MINIMAX SEARCH

In [4]:
# Creating the minimax algorithm

def minimax(state, depth):
    return max_value(state, depth)
    
def max_value(state, depth):
    value = evaluate(state)
    
    if value or not depth:
        return value, None

    max_eval = float('-inf')
    for action in available_moves_user(state, WHITE):
        cur_eval, _ = min_value(get_new_state(state, WHITE, *action), depth - 1)
        
        if cur_eval > max_eval:
            max_eval, movement = cur_eval, action
            
    return max_eval, movement

def min_value(state, depth):
    value = evaluate(state)
    
    if value or not depth:
        return value, None

    min_eval = float('inf')
    for action in available_moves_user(state, BLACK):
        cur_eval, _ = max_value(get_new_state(state, BLACK, *action), depth - 1)
        
        if cur_eval < min_eval:
            min_eval, movement = cur_eval, action
            
    return min_eval, movement

In [5]:
# Testing the minimax search

MINIMAX_DEPTH = 6

grid = np.array([[1, 2, 0],
                 [0, 2, 1],
                 [2, 0, 1]])

evaluation, (start, end) = minimax(grid, MINIMAX_DEPTH)

print('Expected movements:')
print('(0, 1) -> (0, 2)')
print('(2, 0) -> (2, 1)')

print()

print('Evaluation obtained =', evaluation)
print('Movement obtained   =', start, '->', end)

Expected movements:
(0, 1) -> (0, 2)
(2, 0) -> (2, 1)

Evaluation obtained = 1
Movement obtained   = (0, 1) -> (0, 2)


In [6]:
grid = new_grid()

move(grid, BLACK, (2, 0), (1, 1))

evaluation, (start, end) = minimax(grid, MINIMAX_DEPTH)

print('No expected movements:')
print('(2, 2) -> (2, 1)')

print()

print('Evaluation obtained =', evaluation)
print('Movement obtained   =', start, '->', end)

No expected movements:
(2, 2) -> (2, 1)

Evaluation obtained = 0
Movement obtained   = (0, 2) -> (0, 1)


In [7]:
grid = np.array([[2, 1, 0],
                 [0, 2, 1],
                 [0, 2, 1]])

evaluation, (start, end) = minimax(grid, MINIMAX_DEPTH)

print('Expected movements:')
print('(1, 1) -> (0, 2)')

print()

print('Evaluation obtained =', evaluation)
print('Movement obtained   =', start, '->', end)

Expected movements:
(1, 1) -> (0, 2)

Evaluation obtained = 0
Movement obtained   = (1, 1) -> (0, 2)


## GAME PROTOTYPE

In [8]:
grid = TapatanGrid()

user = BLACK
while True:
    clear_output()
    
    print(f'Usuário {user}')
    print(grid)
    print()
    
    if user == BLACK:
        pos_start = tuple(int(c) for c in input(f'Enter the starting position [{user}]='))
        pos_final = tuple(int(c) for c in input(f'Enter the end position      [{user}]='))
    else:
        _, (pos_start, pos_final) = minimax(grid.grid, MINIMAX_DEPTH)
        
    if grid.move(user, pos_start, pos_final):   
        if grid.win(user):
            break
        else:
            user = BLACK if user == WHITE else WHITE

clear_output()

print(f'\n*** User {user} won the match! ***\n')
print(grid)


*** User 2 won the match! ***

2  -  1  -  0
|  \  |  /  |
0  -  2  -  1
|  /  |  \  |
0  -  1  -  2
